# Build the neural network

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call **`model.forward()`** directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.



In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # Executes forward with some other operations
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


# Breakdown

## Model Layers

In [12]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2690,  0.1081, -0.3300,  0.1667,  0.1298, -0.2877, -0.2600,  0.1908,
         -0.0504,  0.1552,  0.1892, -0.3917,  0.1229, -0.0476,  0.7868, -0.4668,
         -0.3941, -0.3454,  0.2391,  0.0563],
        [-0.3443,  0.3801, -0.1544,  0.5486,  0.1395, -0.3368, -0.0185,  0.0119,
         -0.2468,  0.2758,  0.4015, -0.2167,  0.4613,  0.2700,  0.5088, -0.3058,
         -0.4911, -0.0240, -0.0326,  0.3517],
        [-0.0612,  0.2255, -0.3022,  0.5161,  0.2476, -0.4740, -0.1495, -0.0723,
         -0.2133,  0.0211,  0.1948, -0.5316,  0.0849,  0.0050,  0.3703, -0.3264,
         -0.3331, -0.4899,  0.2204,  0.1389]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1081, 0.0000, 0.1667, 0.1298, 0.0000, 0.0000, 0.1908, 0.0000,
         0.1552, 0.1892, 0.0000, 0.1229, 0.0000, 0.7868, 0.0000, 0.0000, 0.0000,
         0.2391, 0.0563],
        [0.0000, 0.3801, 0.0000, 0.5486, 0.1395, 0.0000, 0.0000, 0.0119, 0.0000,
         0.2758, 0.4015, 0.0000, 0.4613, 0.2700, 0.50

## nn.Sequential
nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined.

In [20]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax

In [21]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training.

In [22]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0110,  0.0105, -0.0279,  ...,  0.0206, -0.0313,  0.0332],
        [-0.0020,  0.0171,  0.0118,  ..., -0.0229,  0.0310, -0.0325]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0185, 0.0094], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0413,  0.0307, -0.0034,  ...,  0.0124, -0.0344,  0.0372],
        [ 0.0053,  0.0368,  0.0348,  ..., -0.0274,  0.0416,  0.0319]],
       device='cuda:0', grad_fn=<Sl